In [2]:
from keras.models import Model
from keras.layers import Input, Activation, merge, Dense, Flatten, concatenate
from keras.layers.convolutional import Convolution2D, MaxPooling2D, SeparableConv2D
from keras.layers import BatchNormalization, add, GlobalAveragePooling2D, AveragePooling2D
from keras.optimizers import Adam
from keras.models import load_model
from keras.datasets import cifar10
from keras.utils import np_utils
import numpy as np

Using TensorFlow backend.


In [3]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255.0
X_test /= 255.0

n_classes = 10
Y_train = np_utils.to_categorical(y_train, n_classes)
Y_test = np_utils.to_categorical(y_test, n_classes)

170500096/170498071 [==============================] - 88s 1us/step


In [0]:
def Xception():
        inputs = Input(shape=(32, 32, 3))
    
        # entry flow
       
        x = Convolution2D(32, (3,3), strides=2)(inputs)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Convolution2D(64, (3,3))(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
       
        residual = Convolution2D(128, (1,1), strides=2, padding='same')(x)
        residual = BatchNormalization()(residual)
        
        x = SeparableConv2D(128, (3,3), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = SeparableConv2D(128, (3,3), padding='same')(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D((3, 3), strides=2, padding='same')(x)
        x = add([x, residual])
        
        residual = Convolution2D(256, (1,1), strides=2, padding='same')(x)
        residual = BatchNormalization()(residual)
        
        x = Activation('relu')(x)
        x = SeparableConv2D(256, (3,3), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = SeparableConv2D(256, (3,3), padding='same')(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D((3, 3), strides=2, padding='same')(x)
        x = add([x, residual])
        
        residual = Convolution2D(728, (1,1), strides=2, padding='same')(x)
        residual = BatchNormalization()(residual)
        
        x = Activation('relu')(x)
        x = SeparableConv2D(728, (3,3), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = SeparableConv2D(728, (3,3), padding='same')(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D((3, 3), strides=2, padding='same')(x)
        x = add([x, residual])
        
        # middle flow
        
        for i in range(8):
            residual = x
        
            x = Activation('relu')(x)
            x = SeparableConv2D(728, (3,3), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            x = SeparableConv2D(728, (3,3), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            x = SeparableConv2D(728, (3,3), padding='same')(x)
            x = BatchNormalization()(x)
            x = add([x, residual])
            
        # exit flow
        
        residual = Convolution2D(1024, (1,1), strides=2, padding='same')(x)
        residual = BatchNormalization()(residual)
        
        x = Activation('relu')(x)
        x = SeparableConv2D(728, (3,3), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = SeparableConv2D(1024, (3,3), padding='same')(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D((3, 3), strides=2, padding='same')(x)
        
        x = add([x, residual])
        
        x = SeparableConv2D(1536, (3,3), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = SeparableConv2D(2048, (3,3), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        
        x = GlobalAveragePooling2D()(x)
        x = Dense(10, kernel_initializer='he_normal', activation='softmax')(x)
  
        model = Model(inputs=inputs, outputs=x)
        return model
    

In [9]:
    
model = Xception()
adam = Adam()
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()  

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 15, 15, 32)   896         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_41 (BatchNo (None, 15, 15, 32)   128         conv2d_7[0][0]                   
__________________________________________________________________________________________________
activation_36 (Activation)      (None, 15, 15, 32)   0           batch_normalization_41[0][0]     
__________________________________________________________________________________________________
conv2d_8 (

In [10]:
batch_size = 50
epochs = 10


h = model.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_split=0.2,)

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 136s 3ms/step - loss: 1.4590 - acc: 0.4672 - val_loss: 1.5496 - val_acc: 0.4633
Epoch 2/10
40000/40000 [==============================] - 121s 3ms/step - loss: 1.0562 - acc: 0.6244 - val_loss: 1.6170 - val_acc: 0.5061
Epoch 3/10
40000/40000 [==============================] - 121s 3ms/step - loss: 0.8499 - acc: 0.7020 - val_loss: 1.2297 - val_acc: 0.5949
Epoch 4/10
40000/40000 [==============================] - 121s 3ms/step - loss: 0.7136 - acc: 0.7530 - val_loss: 0.8481 - val_acc: 0.7117
Epoch 5/10
40000/40000 [==============================] - 121s 3ms/step - loss: 0.5935 - acc: 0.7920 - val_loss: 0.8537 - val_acc: 0.7069
Epoch 6/10
40000/40000 [==============================] - 121s 3ms/step - loss: 0.4952 - acc: 0.8278 - val_loss: 0.8569 - val_acc: 0.7172
Epoch 7/10
40000/40000 [==============================] - 120s 3ms/step - loss: 0.4075 - acc: 0.8593 - val_loss: 0.9988 - 